In [4]:
from ase import Atoms
from ase.build import make_supercell
import nglview as nv
import numpy as np

# Lattice constant
a = 5.65

# Define GaAs unit cell
symbols = ['Ga', 'As']
fractional_positions = [
    (0.00, 0.00, 0.00),  # Ga
    (0.25, 0.25, 0.25),  # As
]
cartesian_positions = np.dot(fractional_positions, np.array([
    [a, 0, 0],
    [0, a, 0],
    [0, 0, a]
]))
unitcell = Atoms(symbols=symbols,
                 positions=cartesian_positions,
                 cell=[a, a, a],
                 pbc=True)

# Make a 3x3x3 supercell
P = np.diag([3, 3, 3])  # replication matrix
supercell = make_supercell(unitcell, P)

# Visualize
nv.show_ase(supercell)



NGLWidget()

In [6]:
from ase.build import bulk
import nglview as nv

# Create GaAs zinc blende structure
gaas = bulk('GaAs', crystalstructure='zincblende', a=5.65, cubic=True)

# View interactively with visible bonds
nv.show_ase(gaas)


NGLWidget()

In [8]:
from ase.build import bulk
from ase.build import make_supercell
import nglview as nv
import numpy as np

# Build unit cell
gaas = bulk('GaAs', crystalstructure='zincblende', a=5.65, cubic=True)

# Replicate to get 2×2×2 supercell
supercell = make_supercell(gaas, np.diag([2, 2, 2]))

# Visualize
nv.show_ase(supercell)


NGLWidget()

In [15]:
import numpy as np
from ase import Atoms
from ase.build import bulk
import nglview as nv

# Step 1: Create GaAs structure
a = 5.65
gaas = bulk('GaAs', 'zincblende', a=a, cubic=True)

# Step 2: Define corners of the unit cell
a1, a2, a3 = gaas.cell[0], gaas.cell[1], gaas.cell[2]
origin = np.array([0, 0, 0])
corners = [
    origin,
    a1, a2, a3,
    a1 + a2, a1 + a3, a2 + a3,
    a1 + a2 + a3
]
corners = np.array(corners)

# Step 3: Add dummy atoms at corners (H = hydrogen, only for visual tags)
box_atoms = Atoms(
    symbols=['H'] * 8,
    positions=corners,
    cell=gaas.cell,
    pbc=True
)

# Step 4: Combine GaAs + dummy corner atoms
combined = gaas + box_atoms

# Step 5: Define edges of the unit cell box
corner_offset = len(gaas)  # because corner atoms come after
edges = [
    (0,1), (0,2), (0,3),
    (1,4), (1,5),
    (2,4), (2,6),
    (3,5), (3,6),
    (4,7), (5,7), (6,7)
]
bonds = [(i + corner_offset, j + corner_offset) for i, j in edges]

# Step 6: Set up nglview viewer
view = nv.show_ase(combined)
view.clear_representations()
view.add_ball_and_stick()
view.add_unitcell()

# Step 7: Add unit cell edges as cylinders (gray thin lines)
for i, j in bonds:
    start = combined[i].position.tolist()
    end = combined[j].position.tolist()
    view.shape.add_cylinder(start, end)

view



NGLWidget()

In [17]:
import numpy as np
from ase import Atoms
from ase.build import make_supercell
from ase.io import write
from io import BytesIO
import py3Dmol

# Define lattice constant
a = 5.65

# Base unit cell
ga_frac = [(0,0,0), (0.5,0,0.5), (0,0.5,0.5), (0.5,0.5,0)]
as_frac = [(x+0.25, y+0.25, z+0.25) for x,y,z in ga_frac]
ga_cart = [np.dot(f, np.eye(3)*a) for f in ga_frac]
as_cart = [np.dot(f, np.eye(3)*a) for f in as_frac]
symbols = ['Ga']*4 + ['As']*4
positions = ga_cart + as_cart
gaas = Atoms(symbols=symbols, positions=positions, cell=[a,a,a], pbc=True)

# Supercell
supercell = make_supercell(gaas, np.eye(3)*2)

# Convert to CIF string
cif_io = BytesIO()
write(cif_io, supercell, format='cif')
cif_str = cif_io.getvalue().decode()

# Add box atoms (8 corner dummy atoms, element H)
box_atoms = [
    [0,0,0], [a,0,0], [a,a,0], [0,a,0],
    [0,0,a], [a,0,a], [a,a,a], [0,a,a]
]
xyz_box = [f"{len(box_atoms)}\nBox"]
for pos in box_atoms:
    xyz_box.append(f"H {pos[0]:.3f} {pos[1]:.3f} {pos[2]:.3f}")
xyz_str = "\n".join(xyz_box)

# Set up py3Dmol view
view = py3Dmol.view(width=800, height=800)
view.addModel(cif_str, 'cif')

# Atom styles
view.setStyle({'elem': 'Ga'}, {
    'sphere': {'scale': 0.25, 'color': 'orange'},
    'stick': {'radius': 0.12, 'color': 'orange'}
})
view.setStyle({'elem': 'As'}, {
    'sphere': {'scale': 0.25, 'color': 'purple'},
    'stick': {'radius': 0.12, 'color': 'purple'}
})

# Add bounding box using dummy atoms
view.addModel(xyz_str, "xyz")
view.setStyle({'model': 1}, {'stick': {'radius': 0.05, 'color': 'gray'}})

# Final display
view.setBackgroundColor("white")
view.zoomTo()
view.show()




3Dmol.js failed to load for some reason. Please check your browser console for error messages.